<a href="https://colab.research.google.com/github/peterchang0414/hmm-jax/blob/main/fixed_lag_smoother.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def fixed_lag_smoother(d, alpha, obslik, obsvec, transmt, act):
    '''
    Description...

    Parameters
    ----------
    d         : int
        The desired window length (must be >= 2).
    alpha     : array
        length d window, excluding t0 (columns indexed 1,..., d)
    obslilk   : array
        length d window
    obsvec    : array
        likelihood vector for current observation
    transfmat : array
        transition matrix
    act       : 
        ()

    '''
    return 0